# TFG - Anàlisi de l’abandonament escolar prematur a Catalunya
## Tractament de Dades Any 2020
Jahziel Hidalgo Artigas



## Taula de continguts<a class="anchor" id="index"></a>
* [Imports](#imports)
* [Funcions](#funcions)
* [Càrrega Datasets](#datasets)
* [Comarques](#comarques)
* [Dades Formació](#formacio)
    * [Anàlisi per Categoría](#formacio)
    * [Formació d'Adults](#formacio_adults)
* [Dades Població](#poblacio)
   * [Població per Gènere](#poblacio_genere)
   * [Població per Edat](#poblacio_edat)
   * [Atur per Edat](#atur)
   * [Atur per Gènere](#atur_genere)
   * [Població per Nacionalitat](#poblacio_nacionalitat)
* [Dades Socioeconòmiques](#dades_socioeconomiques)
    * [Repetidors](#repetidors)
    * [Nivell Socioeconòmic (Renda Familiar)](#renta_familiar)
    * [Nivell Socioeconòmic (RFBD](#rfbd)


### Imports<a class="anchor" id="imports"></a>

In [485]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import unicodedata
import os
from unidecode import unidecode

### Funcions<a class="anchor" id="funcions"></a>

In [3]:
def clean_dataframe(df):
    # Netejar les columnes: treure els accents, substituir els espais per guions baixos i convertir a minúscules
    df.columns = [unidecode(col.replace(' ', '_').lower()) for col in df.columns]
    
    # Netejar les dades: treure els accents, convertir a minúscules i processar només les cadenes de text
    df = df.apply(lambda x: x.apply(lambda val: unidecode(str(val)).lower() if isinstance(val, str) else val) 
                  if x.dtype == 'object' else x, axis=0)
    
    return df

def filtrar_comarques(df, columna):
    comarques_valides = [
        'alt camp', 'alt emporda', 'alt penedes', 'alt urgell', 'alta ribagorca', 
        'anoia', 'aran', 'bages', 'baix camp', 'baix ebre', 'baix emporda', 
        'baix llobregat', 'baix penedes', 'barcelones', 'bergueda', 'cerdanya', 
        'conca de barbera', 'garraf', 'garrigues', 'garrotxa', 'girones', 'maresme', 
        'moianes', 'montsia', 'noguera', 'osona', 'pallars jussa', 'pallars sobira', 
        "pla d'urgell", "pla de l'estany", 'priorat', "ribera d'ebre", 'ripolles', 
        'segarra', 'segria', 'selva', 'solsones', 'tarragones', 'terra alta', 'urgell', 
        'valles occidental', 'valles oriental'
    ]
    
    # Filtrar les files que continguin valors en la columna dins de la llista de comarques vàlides
    df = df[df[columna].isin(comarques_valides)].reset_index(drop=True)
    
    return df


### Càrrega Datasets<a class="anchor" id="datasets"></a>

In [183]:
df_comarques = pd.read_csv('csv/2020/comarques.csv', delimiter=';', skiprows=3) # Filtar per Nivell = 'Comarca'
df_formacio = pd.read_csv('csv/2020/nivell_formacio_2020.csv', delimiter=';', skiprows=6) 
df_formacio_adults = pd.read_csv('csv/2020/formacio_adults_2020.csv', delimiter=';', skiprows=5)
df_poblacio_genere = pd.read_csv('csv/2020/poblacio_genere_2020.csv', delimiter=';')
df_poblacio_edat = pd.read_csv('csv/2020/poblacio_edat_2020.csv', delimiter=';')
df_poblacio_nacionalitat = pd.read_csv('csv/2020/poblacio_nacionalitat_2020.csv', delimiter=';', skiprows=6)
df_repetidors = pd.read_csv('csv/2020/alumnes_repetidors_2020.csv', delimiter=';')
df_renta_familiar = pd.read_csv('csv/2020/renta_familiar2020.csv', delimiter=';', skiprows=6)
df_rfbd = pd.read_csv('csv/2020/RFBD_2020.csv', delimiter=';', skiprows=6)
df_atur_edat = pd.read_csv('csv/2020/atur_edat_2020.csv', delimiter=';', skiprows=6)
df_atur_genere = pd.read_csv('csv/2020/atur_genere_2020.csv', delimiter=';', skiprows=6)

In [431]:
datasets = {
    'Comarques': df_comarques,
    'Formació': df_formacio,
    'Formació Adults': df_formacio_adults,
    'Població Sexe': df_poblacio_genere,
    'Població Edat': df_poblacio_edat,
    'Població Nacionalitat': df_poblacio_nacionalitat,
    'Repetidors': df_repetidors,
    'Renta Familiar': df_renta_familiar,
    'RFBD': df_rfbd
}

# Iterar sobre el diccionari per mostrar les primeres files de cada dataset
for name, df in datasets.items():
    print(f"Primeres files de {name}:")
    display(df.head()) 
    print("\n") 

Primeres files de Comarques:


,Nivell,Codi,Nom
0,Comarca,1,Alt Camp
1,Municipi,430017,Aiguamúrcia
2,Municipi,430056,Alcover
3,Municipi,430108,Alió
4,Municipi,430347,Bràfim




Primeres files de Formació:


,comarca,analfabets,educacio_primaria_incompleta,educacio_primaria,primera_etapa_d'educacio_secundaria_i_similar,segona_etapa_d'educacio_secundaria_amb_orientacio_general,segona_etapa_d'educacio_secundaria_amb_orientacio_professional,ensenyament_de_grau_superior_de_formacio_professional_i_similars,graus_universitaris_de_fins_a_240_credits_i_similars,graus_universitaris_de_mes_de_240_credits_i_similars,master_universitari_i_similars,doctorat_universitari,total
0,Alt Camp,105,1370,4445,13665,3740,4410,4015,3060,2400,705,220,38135
1,Alt Empordà,335,5810,15730,42440,17880,11620,7180,9130,8030,1830,655,120640
2,Alt Penedès,185,2155,11870,28265,10210,11605,9155,8410,7065,1825,640,91385
3,Alt Urgell,40,650,2085,5315,2640,1860,1485,1605,1355,430,135,17600
4,Alta Ribagorça,10,130,375,885,505,335,315,365,335,65,30,3350




Primeres files de Formació Adults:


...

,codi,comarca,sexe,repetixen
0,01,alt camp,dona,27
1,01,alt camp,home,74
2,02,alt emporda,dona,128
3,02,alt emporda,home,172
4,03,alt penedes,dona,35




Primeres files de Renta Familiar:


,comarca,rfdb_(miles_de_euros),rfdb_por_habitante_(euros),rfdb_por_habitante_(index_catalunya=100_por_habitante)
0,Alt Camp,700228,15550,"88,6"
1,Alt Empordà,1949301,13934,"79,4"
2,Alt Penedès,1831259,16626,"94,7"
3,Alt Urgell,276359,13732,"78,3"
4,Alta Ribagorça,57401,14665,"83,6"




Primeres files de RFBD:


,comarca,recursos._remuneracion_asalariados,recursos._excedente_bruto_explotacion,recursos._prestaciones_sociales,usos._cotizaciones_sociales,usos._impuestos
0,Alt Camp,549547,158967,237283,170612,90319
1,Alt Empordà,1451220,590288,603165,481126,297021
2,Alt Penedès,1514625,432157,585561,461966,260083
3,Alt Urgell,183086,80550,96098,61099,33868
4,Alta Ribagorça,41550,19002,18379,14397,7793


### Comarques<a class="anchor" id="comarques"></a>

L'estudi es farà per comarques, així que necessitem obtenir el codi de les comarques, el qual ens servirà per agrupar el conjunt final.

In [13]:
# Primerament, netegem el dataset per assegurar-nos que no hi hagi problemes amb els noms de les columnes, accents, etc.
df_comarques = clean_dataframe(df_comarques)

In [15]:
df_comarques.head()

,nivell,codi,nom
0,comarca,1,alt camp
1,municipi,430017,aiguamurcia
2,municipi,430056,alcover
3,municipi,430108,alio
4,municipi,430347,brafim


In [17]:
df_comarques.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 990 entries, 0 to 989
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   nivell  990 non-null    object
 1   codi    990 non-null    int64 
 2   nom     990 non-null    object
dtypes: int64(1), object(2)
memory usage: 23.3+ KB


In [19]:
# Filtrarem les dades per seleccionar només aquelles files on el 'nivell' sigui 'comarca',
# i només seleccionarem les columnes 'codi' i 'nom' que seran necessàries per a l'anàlisi posterior.
df_comarques = df_comarques[df_comarques['nivell'] == 'comarca'][['codi', 'nom']].reset_index(drop=True)
# Convertim la columna 'codi' a format de dos dígits
df_comarques['codi'] = df_comarques['codi'].astype(str).str.zfill(2)
df_comarques.head()

,codi,nom
0,01,alt camp
1,02,alt emporda
2,03,alt penedes
3,04,alt urgell
4,05,alta ribagorca


In [21]:
# Afegim manualment el codi d'Aran (39)
df_comarques.loc[len(df_comarques)] = ['39', 'aran']
df_comarques = df_comarques.sort_values(by='codi').reset_index(drop=True)
df_comarques.tail()

,codi,nom
38,39,aran
39,40,valles occidental
40,41,valles oriental
41,42,moianes
42,43,llucanes


In [23]:
df_comarques['codi'] = df_comarques['codi'].astype(str)
df_comarques.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43 entries, 0 to 42
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   codi    43 non-null     object
 1   nom     43 non-null     object
dtypes: object(2)
memory usage: 820.0+ bytes


In [25]:
# Comprovem els noms únics de les comarques per assegurar-nos que no hi ha comarques duplicades
df_comarques['nom'].unique()

array(['alt camp', 'alt emporda', 'alt penedes', 'alt urgell',
       'alta ribagorca', 'anoia', 'bages', 'baix camp', 'baix ebre',
       'baix emporda', 'baix llobregat', 'baix penedes', 'barcelones',
       'bergueda', 'cerdanya', 'conca de barbera', 'garraf', 'garrigues',
       'garrotxa', 'girones', 'maresme', 'montsia', 'noguera', 'osona',
       'pallars jussa', 'pallars sobira', "pla d'urgell",
       "pla de l'estany", 'priorat', "ribera d'ebre", 'ripolles',
       'segarra', 'segria', 'selva', 'solsones', 'tarragones',
       'terra alta', 'urgell', 'aran', 'valles occidental',
       'valles oriental', 'moianes', 'llucanes'], dtype=object)

In [27]:
# Imprimim el nombre de comarques úniques que tenim en el dataset
print(f"Hi han {len(df_comarques['nom'].unique())} comarques")

Hi han 43 comarques


In [29]:
# Crear el diccionari de codis (comarca -> codi)
comarques_dict = df_comarques.set_index('nom')['codi'].to_dict()
comarques_dict

{'alt camp': '01',
 'alt emporda': '02',
 'alt penedes': '03',
 'alt urgell': '04',
 'alta ribagorca': '05',
 'anoia': '06',
 'bages': '07',
 'baix camp': '08',
 'baix ebre': '09',
 'baix emporda': '10',
 'baix llobregat': '11',
 'baix penedes': '12',
 'barcelones': '13',
 'bergueda': '14',
 'cerdanya': '15',
 'conca de barbera': '16',
 'garraf': '17',
 'garrigues': '18',
 'garrotxa': '19',
 'girones': '20',
 'maresme': '21',
 'montsia': '22',
 'noguera': '23',
 'osona': '24',
 'pallars jussa': '25',
 'pallars sobira': '26',
 "pla d'urgell": '27',
 "pla de l'estany": '28',
 'priorat': '29',
 "ribera d'ebre": '30',
 'ripolles': '31',
 'segarra': '32',
 'segria': '33',
 'selva': '34',
 'solsones': '35',
 'tarragones': '36',
 'terra alta': '37',
 'urgell': '38',
 'aran': '39',
 'valles occidental': '40',
 'valles oriental': '41',
 'moianes': '42',
 'llucanes': '43'}

### Dades Formació<a class="anchor" id="formacio"></a>

#### Anàlisi per Categoría<a class="anchor" id="formacio"></a>

In [465]:
df_formacio = pd.read_csv('csv/2020/nivell_formacio_2020.csv', delimiter=';', skiprows=6) 

In [467]:
# Netejar el dataset per assegurar que les columnes siguin coherents
df_formacio_clean = clean_dataframe(df_formacio)
df_formacio_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55 entries, 0 to 54
Data columns (total 13 columns):
 #   Column                                                            Non-Null Count  Dtype 
---  ------                                                            --------------  ----- 
 0   comarca                                                           55 non-null     object
 1   analfabets                                                        55 non-null     int64 
 2   educacio_primaria_incompleta                                      55 non-null     int64 
 3   educacio_primaria                                                 55 non-null     int64 
 4   primera_etapa_d'educacio_secundaria_i_similar                     55 non-null     int64 
 5   segona_etapa_d'educacio_secundaria_amb_orientacio_general         55 non-null     int64 
 6   segona_etapa_d'educacio_secundaria_amb_orientacio_professional    55 non-null     int64 
 7   ensenyament_de_grau_superior_de_formacio_profe

In [469]:
# Filtrar per comarques
df_formacio_clean = filtrar_comarques(df_formacio_clean, 'comarca')

In [471]:
# Comprovem els canvis realitzats
df_formacio_clean.info()
display(df_formacio_clean.head(10))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 13 columns):
 #   Column                                                            Non-Null Count  Dtype 
---  ------                                                            --------------  ----- 
 0   comarca                                                           42 non-null     object
 1   analfabets                                                        42 non-null     int64 
 2   educacio_primaria_incompleta                                      42 non-null     int64 
 3   educacio_primaria                                                 42 non-null     int64 
 4   primera_etapa_d'educacio_secundaria_i_similar                     42 non-null     int64 
 5   segona_etapa_d'educacio_secundaria_amb_orientacio_general         42 non-null     int64 
 6   segona_etapa_d'educacio_secundaria_amb_orientacio_professional    42 non-null     int64 
 7   ensenyament_de_grau_superior_de_formacio_profe

,comarca,analfabets,educacio_primaria_incompleta,educacio_primaria,primera_etapa_d'educacio_secundaria_i_similar,segona_etapa_d'educacio_secundaria_amb_orientacio_general,segona_etapa_d'educacio_secundaria_amb_orientacio_professional,ensenyament_de_grau_superior_de_formacio_professional_i_similars,graus_universitaris_de_fins_a_240_credits_i_similars,graus_universitaris_de_mes_de_240_credits_i_similars,master_universitari_i_similars,doctorat_universitari,total
0,alt camp,105,1370,4445,13665,3740,4410,4015,3060,2400,705,220,38135
1,alt emporda,335,5810,15730,42440,17880,11620,7180,9130,8030,1830,655,120640
2,alt penedes,185,2155,11870,28265,10210,11605,9155,8410,7065,1825,640,91385
3,alt urgell,40,650,2085,5315,2640,1860,1485,1605,1355,430,135,17600
4,alta ribagorca,10,130,375,885,505,335,315,365,335,65,30,3350
5,anoia,340,3180,11920,35455,11460,13035,10460,8320,6590,1795,490,103055
6,aran,10,125,665,2575,1445,1095,910,990,785,175,45,8820
7,bages,360,4375,19750,50460,16845,17125,14000,14195,11535,3035,905,152580
8,baix camp,1240,5960,17960,52250,20365,17655,15125,14080,12810,3925,1220,162590
9,baix ebre,140,3050,9595,23610,7830,7390,5260,4925,3890,1435,320,67445


In [473]:
# Definim el mapeig de les categories educatives
cat = {
    'analfabetisme_total': ['analfabets'],
    'educacio primaria incompleta_total': ['educacio_primaria_incompleta'],
    'educacio primaria_total': ['educacio_primaria'],
    'educacio secundaria_total': ['primera_etapa_d\'educacio_secundaria_i_similar', 
                            'segona_etapa_d\'educacio_secundaria_amb_orientacio_general',
                            'segona_etapa_d\'educacio_secundaria_amb_orientacio_professional'],
    'formacio professional_total': ['ensenyament_de_grau_superior_de_formacio_professional_i_similars'],
    'graus universitaris_total': ['graus_universitaris_de_fins_a_240_credits_i_similars', 
                            'graus_universitaris_de_mes_de_240_credits_i_similars'],
    'masters universitaris_total': ['master_universitari_i_similars'],
    'doctorat universitari_total': ['doctorat_universitari']
}

# Creem un nou DataFrame per emmagatzemar els resultats
df_grouped = pd.DataFrame()

# Iterem sobre les categories i sumem les columnes corresponents
for categoria, columnas in cat.items():
    df_grouped[categoria] = df_formacio_clean[columnas].sum(axis=1)

# Afegim una nova columna amb el total per comarca, sumant les columnes anteriors
df_grouped['total'] = df_grouped.sum(axis=1) 

# Mostrar el DataFrame resultante con las nuevas columnas de categorías
df_grouped.head()

,analfabetisme_total,educacio primaria incompleta_total,educacio primaria_total,educacio secundaria_total,formacio professional_total,graus universitaris_total,masters universitaris_total,doctorat universitari_total,total
0,105,1370,4445,21815,4015,5460,705,220,38135
1,335,5810,15730,71940,7180,17160,1830,655,120640
2,185,2155,11870,50080,9155,15475,1825,640,91385
3,40,650,2085,9815,1485,2960,430,135,17600
4,10,130,375,1725,315,700,65,30,3350


In [475]:
# Merge entre df_grouped i df_formacio_clean per afegir la columna 'comarca' de df_formacio_clean
df_formacio_clean = df_grouped.merge(df_formacio_clean[['comarca']], left_index=True, right_index=True, how='left')
df_formacio_clean.head()

,analfabetisme_total,educacio primaria incompleta_total,educacio primaria_total,educacio secundaria_total,formacio professional_total,graus universitaris_total,masters universitaris_total,doctorat universitari_total,total,comarca
0,105,1370,4445,21815,4015,5460,705,220,38135,alt camp
1,335,5810,15730,71940,7180,17160,1830,655,120640,alt emporda
2,185,2155,11870,50080,9155,15475,1825,640,91385,alt penedes
3,40,650,2085,9815,1485,2960,430,135,17600,alt urgell
4,10,130,375,1725,315,700,65,30,3350,alta ribagorca


In [477]:
# Reordenem les columnes per posar 'comarca' al principi
df_formacio_clean = df_formacio_clean[['comarca'] + [col for col in df_grouped.columns if col != 'comarca']]
df_formacio_clean.head()

,comarca,analfabetisme_total,educacio primaria incompleta_total,educacio primaria_total,educacio secundaria_total,formacio professional_total,graus universitaris_total,masters universitaris_total,doctorat universitari_total,total
0,alt camp,105,1370,4445,21815,4015,5460,705,220,38135
1,alt emporda,335,5810,15730,71940,7180,17160,1830,655,120640
2,alt penedes,185,2155,11870,50080,9155,15475,1825,640,91385
3,alt urgell,40,650,2085,9815,1485,2960,430,135,17600
4,alta ribagorca,10,130,375,1725,315,700,65,30,3350


In [479]:
# Afegim la columna 'codi' des de df_comarques per fer match amb els noms de les comarques
df_formacio_clean.insert(0, 'codi', df_formacio_clean['comarca'].map(comarques_dict))
df_formacio_clean = df_formacio_clean.sort_values(by='codi').reset_index(drop=True)
df_formacio_clean.head()

,codi,comarca,analfabetisme_total,educacio primaria incompleta_total,educacio primaria_total,educacio secundaria_total,formacio professional_total,graus universitaris_total,masters universitaris_total,doctorat universitari_total,total
0,01,alt camp,105,1370,4445,21815,4015,5460,705,220,38135
1,02,alt emporda,335,5810,15730,71940,7180,17160,1830,655,120640
2,03,alt penedes,185,2155,11870,50080,9155,15475,1825,640,91385
3,04,alt urgell,40,650,2085,9815,1485,2960,430,135,17600
4,05,alta ribagorca,10,130,375,1725,315,700,65,30,3350


In [481]:
df_formacio_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 11 columns):
 #   Column                              Non-Null Count  Dtype 
---  ------                              --------------  ----- 
 0   codi                                42 non-null     object
 1   comarca                             42 non-null     object
 2   analfabetisme_total                 42 non-null     int64 
 3   educacio primaria incompleta_total  42 non-null     int64 
 4   educacio primaria_total             42 non-null     int64 
 5   educacio secundaria_total           42 non-null     int64 
 6   formacio professional_total         42 non-null     int64 
 7   graus universitaris_total           42 non-null     int64 
 8   masters universitaris_total         42 non-null     int64 
 9   doctorat universitari_total         42 non-null     int64 
 10  total                               42 non-null     int64 
dtypes: int64(9), object(2)
memory usage: 3.7+ KB


In [483]:
df_formacio_clean.to_csv('clean_data/2020/df_formacio.csv', index=False)

#### Formació d'Adults<a class="anchor" id="formacio_adults"></a>

In [115]:
df_formacio_adults = pd.read_csv('csv/2020/formacio_adults_2020.csv', delimiter=';', skiprows=5)

In [117]:
df_formacio_adults.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43 entries, 0 to 42
Data columns (total 15 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   comarca                    43 non-null     object
 1   Centres                    43 non-null     int64 
 2   Personal docent            43 non-null     int64 
 3   Alumnes (homes). 16-24     43 non-null     int64 
 4   Alumnes (homes). 25-64     43 non-null     int64 
 5   Alumnes (homes). 65 i més  43 non-null     int64 
 6   Alumnes (homes). Total     43 non-null     int64 
 7   Alumnes (dones). 16-24     43 non-null     int64 
 8   Alumnes (dones). 25-64     43 non-null     int64 
 9   Alumnes (dones). 65 i més  43 non-null     int64 
 10  Alumnes (dones). Total     43 non-null     int64 
 11  Alumnes (total). 16-24     43 non-null     int64 
 12  Alumnes (total). 25-64     43 non-null     int64 
 13  Alumnes (total). 65 i més  43 non-null     int64 
 14  Alumnes (tot

In [119]:
df_formacio_adults.head(10)

,comarca,Centres,Personal docent,Alumnes (homes). 16-24,Alumnes (homes). 25-64,Alumnes (homes). 65 i més,Alumnes (homes). Total,Alumnes (dones). 16-24,Alumnes (dones). 25-64,Alumnes (dones). 65 i més,Alumnes (dones). Total,Alumnes (total). 16-24,Alumnes (total). 25-64,Alumnes (total). 65 i més,Alumnes (total). Total
0,Alt Camp,1,7,22,46,4,72,18,88,17,123,40,134,21,195
1,Alt Empordà,4,36,212,593,15,820,96,256,19,371,308,849,34,1191
2,Alt Penedès,2,10,40,85,8,133,32,100,17,149,72,185,25,282
3,Alt Urgell,1,6,32,64,17,113,13,133,30,176,45,197,47,289
4,Alta Ribagorça,1,1,3,5,0,8,7,19,0,26,10,24,0,34
5,Anoia,1,4,18,51,2,71,13,52,1,66,31,103,3,137
6,Aran,1,3,3,39,1,43,9,94,7,110,12,133,8,153
7,Bages,3,35,101,597,25,723,77,294,26,397,178,891,51,1120
8,Baix Camp,4,41,157,248,26,431,154,453,49,656,311,701,75,1087
9,Baix Ebre,2,17,63,74,19,156,62,188,30,280,125,262,49,436


In [121]:
# Netejar el dataset per assegurar que les columnes siguin coherents
df_formacio_adults_clean = clean_dataframe(df_formacio_adults)
df_formacio_adults_clean.head()

,comarca,centres,personal_docent,alumnes_(homes)._16-24,alumnes_(homes)._25-64,alumnes_(homes)._65_i_mes,alumnes_(homes)._total,alumnes_(dones)._16-24,alumnes_(dones)._25-64,alumnes_(dones)._65_i_mes,alumnes_(dones)._total,alumnes_(total)._16-24,alumnes_(total)._25-64,alumnes_(total)._65_i_mes,alumnes_(total)._total
0,alt camp,1,7,22,46,4,72,18,88,17,123,40,134,21,195
1,alt emporda,4,36,212,593,15,820,96,256,19,371,308,849,34,1191
2,alt penedes,2,10,40,85,8,133,32,100,17,149,72,185,25,282
3,alt urgell,1,6,32,64,17,113,13,133,30,176,45,197,47,289
4,alta ribagorca,1,1,3,5,0,8,7,19,0,26,10,24,0,34


In [123]:
# Filtrar per comarques
df_formacio_adults_clean = filtrar_comarques(df_formacio_adults_clean, 'comarca')
df_formacio_adults_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 15 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   comarca                    42 non-null     object
 1   centres                    42 non-null     int64 
 2   personal_docent            42 non-null     int64 
 3   alumnes_(homes)._16-24     42 non-null     int64 
 4   alumnes_(homes)._25-64     42 non-null     int64 
 5   alumnes_(homes)._65_i_mes  42 non-null     int64 
 6   alumnes_(homes)._total     42 non-null     int64 
 7   alumnes_(dones)._16-24     42 non-null     int64 
 8   alumnes_(dones)._25-64     42 non-null     int64 
 9   alumnes_(dones)._65_i_mes  42 non-null     int64 
 10  alumnes_(dones)._total     42 non-null     int64 
 11  alumnes_(total)._16-24     42 non-null     int64 
 12  alumnes_(total)._25-64     42 non-null     int64 
 13  alumnes_(total)._65_i_mes  42 non-null     int64 
 14  alumnes_(tot

In [125]:
# Suma d'alumnes majors de 25 per a homes, dones i el total
df_formacio_adults_clean.loc[:, 'alumnes_homes_+25_anys'] = df_formacio_adults_clean['alumnes_(homes)._25-64'] + df_formacio_adults['alumnes_(homes)._65_i_mes']
df_formacio_adults_clean.loc[:, 'alumnes_dones_+25_anys'] = df_formacio_adults_clean['alumnes_(dones)._25-64'] + df_formacio_adults['alumnes_(dones)._65_i_mes']
df_formacio_adults_clean.loc[:, 'alumnes_total_+25_anys'] = df_formacio_adults_clean['alumnes_(total)._25-64'] + df_formacio_adults['alumnes_(total)._65_i_mes']


# Convertir les columnes +25 anys a tipus int
df_formacio_adults_clean[['alumnes_homes_+25_anys', 'alumnes_dones_+25_anys', 'alumnes_total_+25_anys']] = (
    df_formacio_adults_clean[['alumnes_homes_+25_anys', 'alumnes_dones_+25_anys', 'alumnes_total_+25_anys']].astype(int)
)

# Seleccionar i renombrar columnes
df_formacio_adults_clean = df_formacio_adults_clean[['comarca',
                                         'alumnes_(homes)._16-24',
                                         'alumnes_(dones)._16-24',
                                         'alumnes_(total)._16-24',
                                         'alumnes_homes_+25_anys',
                                         'alumnes_dones_+25_anys',
                                         'alumnes_total_+25_anys']].rename(columns={
    'alumnes_(homes)._16-24': 'alumnes_homes_16_24_anys',
    'alumnes_(dones)._16-24': 'alumnes_dones_16_24_anys',
    'alumnes_(total)._16-24': 'alumnes_total_16_24_anys'
})
# Dataset resultant
df_formacio_adults_clean.head()


,comarca,alumnes_homes_16_24_anys,alumnes_dones_16_24_anys,alumnes_total_16_24_anys,alumnes_homes_+25_anys,alumnes_dones_+25_anys,alumnes_total_+25_anys
0,alt camp,22,18,40,50,105,155
1,alt emporda,212,96,308,608,275,883
2,alt penedes,40,32,72,93,117,210
3,alt urgell,32,13,45,81,163,244
4,alta ribagorca,3,7,10,5,19,24


In [127]:
df_formacio_adults_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 7 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   comarca                   42 non-null     object
 1   alumnes_homes_16_24_anys  42 non-null     int64 
 2   alumnes_dones_16_24_anys  42 non-null     int64 
 3   alumnes_total_16_24_anys  42 non-null     int64 
 4   alumnes_homes_+25_anys    42 non-null     int32 
 5   alumnes_dones_+25_anys    42 non-null     int32 
 6   alumnes_total_+25_anys    42 non-null     int32 
dtypes: int32(3), int64(3), object(1)
memory usage: 1.9+ KB


In [129]:
# Afegim la columna 'codi' des de df_comarques per fer match amb els noms de les comarques
df_formacio_adults_clean.insert(0, 'codi', df_formacio_adults_clean['comarca'].map(comarques_dict))
df_formacio_adults_clean = df_formacio_adults_clean.sort_values(by='codi').reset_index(drop=True)
df_formacio_adults_clean.head()

,codi,comarca,alumnes_homes_16_24_anys,alumnes_dones_16_24_anys,alumnes_total_16_24_anys,alumnes_homes_+25_anys,alumnes_dones_+25_anys,alumnes_total_+25_anys
0,01,alt camp,22,18,40,50,105,155
1,02,alt emporda,212,96,308,608,275,883
2,03,alt penedes,40,32,72,93,117,210
3,04,alt urgell,32,13,45,81,163,244
4,05,alta ribagorca,3,7,10,5,19,24


In [133]:
df_formacio_adults_clean.to_csv('clean_data/2020/df_formacio_adults.csv', index=False)

### Dades Població<a class="anchor" id="poblacio"></a>

#### Població per Gènere<a class="anchor" id="poblacio_genere"></a>

In [487]:
df_poblacio_genere = pd.read_csv('csv/2020/poblacio_genere_2020.csv', delimiter=';') 

In [489]:
df_poblacio_genere.head()

,comarca,dones,homes,total
0,Alt Camp,22027,22706,44733
1,Alt Empordà,70677,71351,142028
2,Alt Penedès,54846,54656,109502
3,Alt Urgell,10072,10114,20186
4,Alta Ribagorça,1862,1979,3841


In [491]:
# Netejar el dataset per assegurar que les columnes siguin coherents
df_poblacio_genere_clean = clean_dataframe(df_poblacio_genere)
df_poblacio_genere_clean.head()

,comarca,dones,homes,total
0,alt camp,22027,22706,44733
1,alt emporda,70677,71351,142028
2,alt penedes,54846,54656,109502
3,alt urgell,10072,10114,20186
4,alta ribagorca,1862,1979,3841


In [493]:
df_poblacio_genere_clean['comarca'].unique()

array(['alt camp', 'alt emporda', 'alt penedes', 'alt urgell',
       'alta ribagorca', 'anoia', 'aran', 'bages', 'baix camp',
       'baix ebre', 'baix emporda', 'baix llobregat', 'baix penedes',
       'barcelones', 'bergueda', 'cerdanya', 'conca de barbera', 'garraf',
       'garrigues', 'garrotxa', 'girones', 'maresme', 'moianes',
       'montsia', 'noguera', 'osona', 'pallars jussa', 'pallars sobira',
       "pla d'urgell", "pla de l'estany", 'priorat', "ribera d'ebre",
       'ripolles', 'segarra', 'segria', 'selva', 'solsones', 'tarragones',
       'terra alta', 'urgell', 'valles occidental', 'valles oriental',
       'catalunya', 'metropolita', 'comarques gironines',
       'camp de tarragona', "terres de l'ebre", 'ponent',
       'comarques centrals', 'alt pirineu i aran', 'penedes', 'barcelona',
       'girona', 'lleida', 'tarragona'], dtype=object)

In [495]:
# Filtrem per comarques
df_poblacio_genere_clean = filtrar_comarques(df_poblacio_genere_clean, 'comarca')

In [497]:
df_poblacio_genere_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   comarca  42 non-null     object
 1   dones    42 non-null     int64 
 2   homes    42 non-null     int64 
 3   total    42 non-null     int64 
dtypes: int64(3), object(1)
memory usage: 1.4+ KB


In [499]:
df_poblacio_genere_clean.head()

,comarca,dones,homes,total
0,alt camp,22027,22706,44733
1,alt emporda,70677,71351,142028
2,alt penedes,54846,54656,109502
3,alt urgell,10072,10114,20186
4,alta ribagorca,1862,1979,3841


In [501]:
# Afegim la columna 'codi' des de df_comarques per fer match amb els noms de les comarques
df_poblacio_genere_clean.insert(0, 'codi', df_poblacio_genere_clean['comarca'].map(comarques_dict))
df_poblacio_genere_clean = df_poblacio_genere_clean.sort_values(by='codi').reset_index(drop=True)
df_poblacio_genere_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   codi     42 non-null     object
 1   comarca  42 non-null     object
 2   dones    42 non-null     int64 
 3   homes    42 non-null     int64 
 4   total    42 non-null     int64 
dtypes: int64(3), object(2)
memory usage: 1.8+ KB


In [503]:
df_poblacio_genere_clean.head()

,codi,comarca,dones,homes,total
0,01,alt camp,22027,22706,44733
1,02,alt emporda,70677,71351,142028
2,03,alt penedes,54846,54656,109502
3,04,alt urgell,10072,10114,20186
4,05,alta ribagorca,1862,1979,3841


In [505]:
df_poblacio_genere_clean.to_csv('clean_data/2020/df_poblacio_genere.csv', index=False)

#### Població per Edat<a class="anchor" id="poblacio_edat"></a>

In [507]:
df_poblacio_edat = pd.read_csv('csv/2020/poblacio_edat_2020.csv', delimiter=';')

In [509]:
df_poblacio_edat.head()

,comarca,poblacio_total_16_a_24_anys,poblacio_homes_16_a_24_anys,poblacio_dones_16_a_24_anys
0,Alt Camp,4176,2195,1981
1,Alt Empordà,13092,6756,6336
2,Alt Penedès,10094,5160,4934
3,Alt Urgell,1678,882,796
4,Alta Ribagorça,277,134,143


In [511]:
df_poblacio_edat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55 entries, 0 to 54
Data columns (total 4 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   comarca                      55 non-null     object
 1   poblacio_total_16_a_24_anys  55 non-null     int64 
 2   poblacio_homes_16_a_24_anys  55 non-null     int64 
 3   poblacio_dones_16_a_24_anys  55 non-null     int64 
dtypes: int64(3), object(1)
memory usage: 1.8+ KB


In [513]:
# Netejar el DataFrame 'df_poblacio_edat' utilitzant la funció 'clean_dataframe'
df_poblacio_edat_clean = clean_dataframe(df_poblacio_edat)
df_poblacio_edat_clean.head()

,comarca,poblacio_total_16_a_24_anys,poblacio_homes_16_a_24_anys,poblacio_dones_16_a_24_anys
0,alt camp,4176,2195,1981
1,alt emporda,13092,6756,6336
2,alt penedes,10094,5160,4934
3,alt urgell,1678,882,796
4,alta ribagorca,277,134,143


In [515]:
# Filtrar les comarques utilitzant la funció 'filtrar_comarques'
df_poblacio_edat_clean = filtrar_comarques(df_poblacio_edat_clean, 'comarca')
df_poblacio_edat_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 4 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   comarca                      42 non-null     object
 1   poblacio_total_16_a_24_anys  42 non-null     int64 
 2   poblacio_homes_16_a_24_anys  42 non-null     int64 
 3   poblacio_dones_16_a_24_anys  42 non-null     int64 
dtypes: int64(3), object(1)
memory usage: 1.4+ KB


In [517]:
df_poblacio_edat_clean.head()

,comarca,poblacio_total_16_a_24_anys,poblacio_homes_16_a_24_anys,poblacio_dones_16_a_24_anys
0,alt camp,4176,2195,1981
1,alt emporda,13092,6756,6336
2,alt penedes,10094,5160,4934
3,alt urgell,1678,882,796
4,alta ribagorca,277,134,143


In [519]:
# Afegim la columna 'codi' des de df_comarques per fer match amb els noms de les comarques
df_poblacio_edat_clean.insert(0, 'codi', df_poblacio_edat_clean['comarca'].map(comarques_dict))
df_poblacio_edat_clean = df_poblacio_edat_clean.sort_values(by='codi').reset_index(drop=True)
df_poblacio_edat_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 5 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   codi                         42 non-null     object
 1   comarca                      42 non-null     object
 2   poblacio_total_16_a_24_anys  42 non-null     int64 
 3   poblacio_homes_16_a_24_anys  42 non-null     int64 
 4   poblacio_dones_16_a_24_anys  42 non-null     int64 
dtypes: int64(3), object(2)
memory usage: 1.8+ KB


In [521]:
df_poblacio_edat_clean.to_csv('clean_data/2020/df_poblacio_edat.csv', index=False)

#### Atur per Edat<a class="anchor" id="atur"></a>

In [220]:
df_atur_edat.head()

,comarca,16-24,25+,Total
0,Alt Camp,267.6,2827.0,3094.6
1,Alt Empordà,786.8,7871.1,8657.9
2,Alt Penedès,422.9,5736.7,6159.6
3,Alt Urgell,70.9,894.1,965.0
4,Alta Ribagorça,11.2,126.7,137.9


In [222]:
df_atur_edat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55 entries, 0 to 54
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   comarca  55 non-null     object 
 1   16-24    55 non-null     float64
 2   25+      55 non-null     float64
 3   Total    55 non-null     float64
dtypes: float64(3), object(1)
memory usage: 1.8+ KB


In [224]:
# Netejar el DataFrame 'df_atur_edat' amb la funció 'clean_dataframe'
df_atur_edat_clean = clean_dataframe(df_atur_edat)
df_atur_edat_clean.head()

,comarca,16-24,25+,total
0,alt camp,267.6,2827.0,3094.6
1,alt emporda,786.8,7871.1,8657.9
2,alt penedes,422.9,5736.7,6159.6
3,alt urgell,70.9,894.1,965.0
4,alta ribagorca,11.2,126.7,137.9


In [226]:
# Obtenir els valors únics de la columna 'comarca' en el DataFrame netejat
df_atur_edat_clean['comarca'].unique()

array(['alt camp', 'alt emporda', 'alt penedes', 'alt urgell',
       'alta ribagorca', 'anoia', 'aran', 'bages', 'baix camp',
       'baix ebre', 'baix emporda', 'baix llobregat', 'baix penedes',
       'barcelones', 'bergueda', 'cerdanya', 'conca de barbera', 'garraf',
       'garrigues', 'garrotxa', 'girones', 'maresme', 'moianes',
       'montsia', 'noguera', 'osona', 'pallars jussa', 'pallars sobira',
       "pla d'urgell", "pla de l'estany", 'priorat', "ribera d'ebre",
       'ripolles', 'segarra', 'segria', 'selva', 'solsones', 'tarragones',
       'terra alta', 'urgell', 'valles occidental', 'valles oriental',
       'cataluna', 'metropolita', 'comarques gironines',
       'camp de tarragona', "terres de l'ebre", 'ponent',
       'comarques centrals', 'alt pirineu i aran', 'penedes', 'barcelona',
       'girona', 'lleida', 'tarragona'], dtype=object)

In [228]:
# Filtrar les comarques en el DataFrame netejat
df_atur_edat_clean = filtrar_comarques(df_atur_edat_clean, 'comarca')
# Obtenir el nombre total de comarques úniques després del filtratge
len(df_atur_edat_clean['comarca'].unique())

42

In [230]:
# Afegim la columna 'codi' des de df_comarques per fer match amb els noms de les comarques
df_atur_edat_clean.insert(0, 'codi', df_atur_edat_clean['comarca'].map(comarques_dict))
# Ordenar el DataFrame per la columna 'codi' i reiniciar l'índex per mantenir-lo consecutiu
df_atur_edat_clean = df_atur_edat_clean.sort_values(by='codi').reset_index(drop=True)
df_atur_edat_clean.head()

,codi,comarca,16-24,25+,total
0,01,alt camp,267.6,2827.0,3094.6
1,02,alt emporda,786.8,7871.1,8657.9
2,03,alt penedes,422.9,5736.7,6159.6
3,04,alt urgell,70.9,894.1,965.0
4,05,alta ribagorca,11.2,126.7,137.9


In [232]:
df_atur_edat_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   codi     42 non-null     object 
 1   comarca  42 non-null     object 
 2   16-24    42 non-null     float64
 3   25+      42 non-null     float64
 4   total    42 non-null     float64
dtypes: float64(3), object(2)
memory usage: 1.8+ KB


In [234]:
# Identificar les columnes de tipus float al dataset
columnes_float = df_atur_edat_clean.select_dtypes(include=['float']).columns

# Convertir les columnes de float a int
df_atur_edat_clean[columnes_float] = (
    np.ceil(df_atur_edat_clean[columnes_float]).astype(int)
)
# Verificar el resultat
df_atur_edat_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   codi     42 non-null     object
 1   comarca  42 non-null     object
 2   16-24    42 non-null     int32 
 3   25+      42 non-null     int32 
 4   total    42 non-null     int32 
dtypes: int32(3), object(2)
memory usage: 1.3+ KB


In [236]:
df_atur_edat_clean.head()

,codi,comarca,16-24,25+,total
0,01,alt camp,268,2827,3095
1,02,alt emporda,787,7872,8658
2,03,alt penedes,423,5737,6160
3,04,alt urgell,71,895,965
4,05,alta ribagorca,12,127,138


In [238]:
df_atur_edat_clean.to_csv('clean_data/2020/df_atur_edat.csv', index=False)

#### Atur per Gènere<a class="anchor" id="atur_genere"></a>

In [525]:
df_atur_genere.head()

,comarca,Total_homes,Total_dones,Total
0,Alt Camp,1391.4,1703.1,3094.5
1,Alt Empordà,3885.6,4772.3,8657.9
2,Alt Penedès,2631.8,3527.8,6159.6
3,Alt Urgell,443.2,521.8,965.0
4,Alta Ribagorça,69.1,68.8,137.9


In [527]:
df_atur_genere.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55 entries, 0 to 54
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   comarca      55 non-null     object 
 1   Total_homes  55 non-null     float64
 2   Total_dones  55 non-null     float64
 3   Total        55 non-null     float64
dtypes: float64(3), object(1)
memory usage: 1.8+ KB


In [529]:
# Netejar el DataFrame 'df_atur_genere' amb la funció 'clean_dataframe'
df_atur_genere_clean = clean_dataframe(df_atur_genere)
df_atur_genere_clean.head()

,comarca,total_homes,total_dones,total
0,alt camp,1391.4,1703.1,3094.5
1,alt emporda,3885.6,4772.3,8657.9
2,alt penedes,2631.8,3527.8,6159.6
3,alt urgell,443.2,521.8,965.0
4,alta ribagorca,69.1,68.8,137.9


In [531]:
# Obtenir els valors únics de la columna 'comarca' en el DataFrame netejat
df_atur_genere_clean['comarca'].unique()

array(['alt camp', 'alt emporda', 'alt penedes', 'alt urgell',
       'alta ribagorca', 'anoia', 'aran', 'bages', 'baix camp',
       'baix ebre', 'baix emporda', 'baix llobregat', 'baix penedes',
       'barcelones', 'bergueda', 'cerdanya', 'conca de barbera', 'garraf',
       'garrigues', 'garrotxa', 'girones', 'maresme', 'moianes',
       'montsia', 'noguera', 'osona', 'pallars jussa', 'pallars sobira',
       "pla d'urgell", "pla de l'estany", 'priorat', "ribera d'ebre",
       'ripolles', 'segarra', 'segria', 'selva', 'solsones', 'tarragones',
       'terra alta', 'urgell', 'valles occidental', 'valles oriental',
       'cataluna', 'metropolita', 'comarques gironines',
       'camp de tarragona', "terres de l'ebre", 'ponent',
       'comarques centrals', 'alt pirineu i aran', 'penedes', 'barcelona',
       'girona', 'lleida', 'tarragona'], dtype=object)

In [533]:
# Filtrar les comarques en el DataFrame netejat
df_atur_genere_clean = filtrar_comarques(df_atur_genere_clean, 'comarca')
# Obtenir el nombre total de comarques úniques després del filtratge
len(df_atur_genere_clean['comarca'].unique())

42

In [535]:
# Afegim la columna 'codi' des de df_comarques per fer match amb els noms de les comarques
df_atur_genere_clean.insert(0, 'codi', df_atur_genere_clean['comarca'].map(comarques_dict))
# Ordenar el DataFrame per la columna 'codi' i reiniciar l'índex per mantenir-lo consecutiu
df_atur_genere_clean = df_atur_genere_clean.sort_values(by='codi').reset_index(drop=True)
df_atur_genere_clean.head()

,codi,comarca,total_homes,total_dones,total
0,01,alt camp,1391.4,1703.1,3094.5
1,02,alt emporda,3885.6,4772.3,8657.9
2,03,alt penedes,2631.8,3527.8,6159.6
3,04,alt urgell,443.2,521.8,965.0
4,05,alta ribagorca,69.1,68.8,137.9


In [537]:
df_atur_genere_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   codi         42 non-null     object 
 1   comarca      42 non-null     object 
 2   total_homes  42 non-null     float64
 3   total_dones  42 non-null     float64
 4   total        42 non-null     float64
dtypes: float64(3), object(2)
memory usage: 1.8+ KB


In [539]:
# Identificar les columnes de tipus float al dataset
columnes_float = df_atur_genere_clean.select_dtypes(include=['float']).columns

# Convertir les columnes de float a int
df_atur_genere_clean[columnes_float] = (
    np.ceil(df_atur_genere_clean[columnes_float]).astype(int)
)
# Verificar el resultat
df_atur_genere_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   codi         42 non-null     object
 1   comarca      42 non-null     object
 2   total_homes  42 non-null     int32 
 3   total_dones  42 non-null     int32 
 4   total        42 non-null     int32 
dtypes: int32(3), object(2)
memory usage: 1.3+ KB


In [541]:
df_atur_genere_clean.head()

,codi,comarca,total_homes,total_dones,total
0,01,alt camp,1392,1704,3095
1,02,alt emporda,3886,4773,8658
2,03,alt penedes,2632,3528,6160
3,04,alt urgell,444,522,965
4,05,alta ribagorca,70,69,138


In [543]:
df_atur_genere_clean.to_csv('clean_data/2020/df_atur_genere.csv', index=False)

In [ ]:
df_poblacio_inactiva_merged.info()

In [ ]:
# Crear un nou DataFrame amb les columnes 'codi', 'comarca', 'poblacio_estudiant' i 'total_poblacio'
df_pct_ratio_estudiants = df_poblacio_inactiva_merged[['codi', 'comarca', 'poblacio_estudiant', 'total_poblacio']].copy()

# Calcular el percentatge d'estudiants respecte al total de població
df_pct_ratio_estudiants['pct_estudiants'] = (df_poblacio_inactiva_merged['poblacio_estudiant'] / df_poblacio_inactiva_merged['total_poblacio'] * 100).round(2)

# Calcular la ràtio d'estudiants respecte al total de població
df_pct_ratio_estudiants['ratio_estudiants'] = (df_poblacio_inactiva_merged['poblacio_estudiant'] / df_poblacio_inactiva_merged['total_poblacio'])
df_pct_ratio_estudiants.head()

In [ ]:
df_pct_ratio_estudiants.info()

In [ ]:
df_pct_ratio_estudiants.to_csv('clean_data/2020/df_estudiants_perc_ratio.csv', index=False)

#### Població per Nacionalitat<a class="anchor" id="poblacio_nacionalitat"></a>

In [270]:
df_poblacio_nacionalitat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55 entries, 0 to 54
Data columns (total 9 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   comarca                     55 non-null     object
 1   Espanyola                   55 non-null     int64 
 2   Resta UE                    55 non-null     int64 
 3   Resta d'Europa              55 non-null     int64 
 4   Àfrica                      55 non-null     int64 
 5   Amèrica del Nord i Central  55 non-null     int64 
 6   Amèrica del Sud             55 non-null     int64 
 7   Àsia i Oceania              55 non-null     int64 
 8   Total                       55 non-null     int64 
dtypes: int64(8), object(1)
memory usage: 4.0+ KB


In [272]:
df_poblacio_nacionalitat.head()

,comarca,Espanyola,Resta UE,Resta d'Europa,Àfrica,Amèrica del Nord i Central,Amèrica del Sud,Àsia i Oceania,Total
0,Alt Camp,38847,1210,482,2978,226,666,286,44695
1,Alt Empordà,107462,11632,2100,13631,2441,4235,1123,142624
2,Alt Penedès,97269,1906,652,6362,681,1935,801,109606
3,Alt Urgell,17692,1004,217,400,173,623,122,20231
4,Alta Ribagorça,3357,169,55,132,28,77,7,3825


In [274]:
# Netejar el DataFrame 'df_poblacio_nacionalitat' amb la funció 'clean_dataframe'
df_poblacio_nacionalitat_clean = clean_dataframe(df_poblacio_nacionalitat)
# Filtrar les comarques en el DataFrame netejat
df_poblacio_nacionalitat_clean = filtrar_comarques(df_poblacio_nacionalitat_clean, 'comarca')
df_poblacio_nacionalitat_clean.head()

,comarca,espanyola,resta_ue,resta_d'europa,africa,america_del_nord_i_central,america_del_sud,asia_i_oceania,total
0,alt camp,38847,1210,482,2978,226,666,286,44695
1,alt emporda,107462,11632,2100,13631,2441,4235,1123,142624
2,alt penedes,97269,1906,652,6362,681,1935,801,109606
3,alt urgell,17692,1004,217,400,173,623,122,20231
4,alta ribagorca,3357,169,55,132,28,77,7,3825


In [276]:
# Afegim la columna 'codi' des de df_comarques per fer match amb els noms de les comarques
df_poblacio_nacionalitat_clean.insert(0, 'codi', df_poblacio_nacionalitat_clean['comarca'].map(comarques_dict))
# Ordenar el DataFrame per la columna 'codi' i reiniciar l'índex per mantenir-lo consecutiu
df_poblacio_nacionalitat_clean = df_poblacio_nacionalitat_clean.sort_values(by='codi').reset_index(drop=True)
df_poblacio_nacionalitat_clean.head()

,codi,comarca,espanyola,resta_ue,resta_d'europa,africa,america_del_nord_i_central,america_del_sud,asia_i_oceania,total
0,01,alt camp,38847,1210,482,2978,226,666,286,44695
1,02,alt emporda,107462,11632,2100,13631,2441,4235,1123,142624
2,03,alt penedes,97269,1906,652,6362,681,1935,801,109606
3,04,alt urgell,17692,1004,217,400,173,623,122,20231
4,05,alta ribagorca,3357,169,55,132,28,77,7,3825


In [278]:
# Llista de nacionalitats
nacionalitats = ['espanyola', 'resta_ue', 'resta_d\'europa', 'africa', 'america_del_nord_i_central', 'america_del_sud', 'asia_i_oceania']

# Calcular el percentatge i la ràtio per a cada nacionalitat
for nacionalitat in nacionalitats:
    # Percentatge respecte al total per comarca
    df_poblacio_nacionalitat_clean[f'{nacionalitat}_pct'] = (
        df_poblacio_nacionalitat_clean[nacionalitat] / df_poblacio_nacionalitat_clean['total'] * 100
    ).round(2)
    
    # Ràtio respecte al total per comarca
    df_poblacio_nacionalitat_clean[f'{nacionalitat}_ratio'] = (
        df_poblacio_nacionalitat_clean[nacionalitat] / df_poblacio_nacionalitat_clean['total']
    )

# Mostrar les primeres files del DataFrame resultant
df_poblacio_nacionalitat_clean.head()

,codi,comarca,espanyola,resta_ue,resta_d'europa,africa,america_del_nord_i_central,america_del_sud,asia_i_oceania,total,...,resta_d'europa_pct,resta_d'europa_ratio,africa_pct,africa_ratio,america_del_nord_i_central_pct,america_del_nord_i_central_ratio,america_del_sud_pct,america_del_sud_ratio,asia_i_oceania_pct,asia_i_oceania_ratio
0,01,alt camp,38847,1210,482,2978,226,666,286,44695,...,1.08,0.010784,6.66,0.066629,0.51,0.005056,1.49,0.014901,0.64,0.006399
1,02,alt emporda,107462,11632,2100,13631,2441,4235,1123,142624,...,1.47,0.014724,9.56,0.095573,1.71,0.017115,2.97,0.029693,0.79,0.007874
2,03,alt penedes,97269,1906,652,6362,681,1935,801,109606,...,0.59,0.005949,5.80,0.058044,0.62,0.006213,1.77,0.017654,0.73,0.007308
3,04,alt urgell,17692,1004,217,400,173,623,122,20231,...,1.07,0.010726,1.98,0.019772,0.86,0.008551,3.08,0.030794,0.60,0.006030
4,05,alta ribagorca,3357,169,55,132,28,77,7,3825,...,1.44,0.014379,3.45,0.034510,0.73,0.007320,2.01,0.020131,0.18,0.001830


In [280]:
df_poblacio_nacionalitat_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 24 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   codi                              42 non-null     object 
 1   comarca                           42 non-null     object 
 2   espanyola                         42 non-null     int64  
 3   resta_ue                          42 non-null     int64  
 4   resta_d'europa                    42 non-null     int64  
 5   africa                            42 non-null     int64  
 6   america_del_nord_i_central        42 non-null     int64  
 7   america_del_sud                   42 non-null     int64  
 8   asia_i_oceania                    42 non-null     int64  
 9   total                             42 non-null     int64  
 10  espanyola_pct                     42 non-null     float64
 11  espanyola_ratio                   42 non-null     float64
 12  resta_ue_p

In [282]:
df_poblacio_nacionalitat_clean.to_csv('clean_data/2020/df_poblacio_nacionalitat.csv', index=False)

### Dades Socioeconòmiques<a class="anchor" id="dades_socioeconomiques"></a>

#### Repetidors<a class="anchor" id="repetidors"></a>

https://educacio.gencat.cat/ca/departament/estadistiques/estadistiques-ensenyament/cursos-anteriors/curs-2021-2022/eso/

In [284]:
df_repetidors.head()

,Curs,Codi àrea territorial,Àrea territorial,Codi comarca,Comarca,Codi municipi,Municipi,Codi districte,Naturalesa,Titularitat,Estudi,Nivell,Sexe,Concert,Resultat de l'avaluació,Nombre alumnes
0,2019/2020,108,CONSORCI D'EDUCACIÓ DE BARCELONA,13,BARCELONÈS,8019,BARCELONA,801901,PÚBLIC,DEPARTAMENT D'ENSENYAMENT,EDUCACIÓ SECUNDÀRIA OBLIGATÒRIA,1,DONA,NaN,Alumnes avaluats,2
1,2019/2020,108,CONSORCI D'EDUCACIÓ DE BARCELONA,13,BARCELONÈS,8019,BARCELONA,801901,PÚBLIC,DEPARTAMENT D'ENSENYAMENT,EDUCACIÓ SECUNDÀRIA OBLIGATÒRIA,1,DONA,NaN,Alumnes avaluats,43
2,2019/2020,108,CONSORCI D'EDUCACIÓ DE BARCELONA,13,BARCELONÈS,8019,BARCELONA,801901,PÚBLIC,DEPARTAMENT D'ENSENYAMENT,EDUCACIÓ SECUNDÀRIA OBLIGATÒRIA,1,DONA,NaN,Alumnes avaluats,83
3,2019/2020,108,CONSORCI D'EDUCACIÓ DE BARCELONA,13,BARCELONÈS,8019,BARCELONA,801901,PÚBLIC,DEPARTAMENT D'ENSENYAMENT,EDUCACIÓ SECUNDÀRIA OBLIGATÒRIA,1,DONA,NaN,No promocionen,2
4,2019/2020,108,CONSORCI D'EDUCACIÓ DE BARCELONA,13,BARCELONÈS,8019,BARCELONA,801901,PÚBLIC,DEPARTAMENT D'ENSENYAMENT,EDUCACIÓ SECUNDÀRIA OBLIGATÒRIA,1,DONA,NaN,Promocionen al mes de juny,83


In [286]:
df_repetidors.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24043 entries, 0 to 24042
Data columns (total 16 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Curs                     24043 non-null  object
 1   Codi àrea territorial    24043 non-null  int64 
 2   Àrea territorial         24043 non-null  object
 3   Codi comarca             24043 non-null  int64 
 4   Comarca                  24043 non-null  object
 5   Codi municipi            24043 non-null  int64 
 6   Municipi                 24043 non-null  object
 7   Codi districte           24043 non-null  int64 
 8   Naturalesa               24043 non-null  object
 9   Titularitat              24043 non-null  object
 10  Estudi                   24043 non-null  object
 11  Nivell                   24043 non-null  int64 
 12  Sexe                     24043 non-null  object
 13  Concert                  8798 non-null   object
 14  Resultat de l'avaluació  24043 non-nul

In [288]:
df_repetidors['Comarca'].unique()

array(['BARCELONÈS', 'ALT EMPORDÀ', 'BAIX EMPORDÀ', 'CERDANYA',
       'GARROTXA', 'GIRONÈS', "PLA DE L'ESTANY", 'RIPOLLÈS', 'SELVA',
       'ALT URGELL', 'ALTA RIBAGORÇA', 'GARRIGUES', 'NOGUERA',
       'PALLARS JUSSÀ', 'PALLARS SOBIRÀ', "PLA D'URGELL", 'SEGARRA',
       'SEGRIÀ', 'URGELL', "VAL D'ARAN", 'ALT CAMP', 'BAIX CAMP',
       'BAIX PENEDÈS', 'CONCA DE BARBERÀ', 'PRIORAT', 'TARRAGONÈS',
       'ALT PENEDÈS', 'GARRAF', 'BAIX EBRE', 'MONTSIÀ', "RIBERA D'EBRE",
       'TERRA ALTA', 'BAIX LLOBREGAT', 'VALLÈS OCCIDENTAL', 'MARESME',
       'VALLÈS ORIENTAL', 'ANOIA', 'BAGES', 'BERGUEDÀ', 'OSONA',
       'SOLSONÈS', 'MOIANÈS'], dtype=object)

In [290]:
# Netejar el DataFrame 'df_repetidors' utilitzant la funció 'clean_dataframe'
df_repetidors_clean = clean_dataframe(df_repetidors)
df_repetidors_clean.head()

,curs,codi_area_territorial,area_territorial,codi_comarca,comarca,codi_municipi,municipi,codi_districte,naturalesa,titularitat,estudi,nivell,sexe,concert,resultat_de_l'avaluacio,nombre_alumnes
0,2019/2020,108,consorci d'educacio de barcelona,13,barcelones,8019,barcelona,801901,public,departament d'ensenyament,educacio secundaria obligatoria,1,dona,NaN,alumnes avaluats,2
1,2019/2020,108,consorci d'educacio de barcelona,13,barcelones,8019,barcelona,801901,public,departament d'ensenyament,educacio secundaria obligatoria,1,dona,NaN,alumnes avaluats,43
2,2019/2020,108,consorci d'educacio de barcelona,13,barcelones,8019,barcelona,801901,public,departament d'ensenyament,educacio secundaria obligatoria,1,dona,NaN,alumnes avaluats,83
3,2019/2020,108,consorci d'educacio de barcelona,13,barcelones,8019,barcelona,801901,public,departament d'ensenyament,educacio secundaria obligatoria,1,dona,NaN,no promocionen,2
4,2019/2020,108,consorci d'educacio de barcelona,13,barcelones,8019,barcelona,801901,public,departament d'ensenyament,educacio secundaria obligatoria,1,dona,NaN,promocionen al mes de juny,83


In [294]:
# Filtrar les dades del DataFrame per obtenir només les comarques rellevants
df_repetidors_clean = filtrar_comarques(df_repetidors_clean, 'comarca')

In [296]:
# Obtenir els valors únics de la columna 'comarca' en el DataFrame netejat
display(df_repetidors_clean['comarca'].unique())
print(len(df_repetidors_clean['comarca'].unique()))

array(['barcelones', 'alt emporda', 'baix emporda', 'cerdanya',
       'garrotxa', 'girones', "pla de l'estany", 'ripolles', 'selva',
       'alt urgell', 'alta ribagorca', 'garrigues', 'noguera',
       'pallars jussa', 'pallars sobira', "pla d'urgell", 'segarra',
       'segria', 'urgell', 'alt camp', 'baix camp', 'baix penedes',
       'conca de barbera', 'priorat', 'tarragones', 'alt penedes',
       'garraf', 'baix ebre', 'montsia', "ribera d'ebre", 'terra alta',
       'baix llobregat', 'valles occidental', 'maresme',
       'valles oriental', 'anoia', 'bages', 'bergueda', 'osona',
       'solsones', 'moianes'], dtype=object)

41


In [298]:
# Obtenir els valors únics de la columna 'resultat_de_l'avaluacio' en el DataFrame netejat
df_repetidors_clean['resultat_de_l\'avaluacio'].unique()

array(['alumnes avaluats', 'no promocionen', 'promocionen al mes de juny',
       'promocionen al mes de setembre', 'promocionen amb pendents',
       "s'ha expedit certificat"], dtype=object)

In [300]:
# Filtrar el DataFrame per mantenir només les files amb 'resultat_de_l'avaluacio' igual a 'no promocionen'
df_no_promocionen = df_repetidors_clean[df_repetidors_clean['resultat_de_l\'avaluacio'] == 'no promocionen']

# Agrupar les dades per 'comarca' i 'sexe', sumant els valors de 'nombre_alumnes'
df_repetidors = df_no_promocionen.groupby(['comarca', 'sexe'])['nombre_alumnes'].sum().reset_index()

# Renombrar la columna 'nombre_alumnes' a 'repetixen'
df_repetidors = df_repetidors.rename(columns={'nombre_alumnes': 'repetixen'})

# Mostrar les primeres files del DataFrame resultant
df_repetidors.head()

,comarca,sexe,repetixen
0,alt camp,dona,27
1,alt camp,home,74
2,alt emporda,dona,128
3,alt emporda,home,172
4,alt penedes,dona,35


In [302]:
# Afegim la columna 'codi'  per fer match amb els noms de les comarques
df_repetidors.insert(0, 'codi', df_repetidors['comarca'].map(comarques_dict))
# Ordenar el DataFrame per la columna 'codi' i reiniciar l'índex per mantenir-lo consecutiu
df_repetidors = df_repetidors.sort_values(by='codi').reset_index(drop=True)
df_repetidors.head()

,codi,comarca,sexe,repetixen
0,01,alt camp,dona,27
1,01,alt camp,home,74
2,02,alt emporda,dona,128
3,02,alt emporda,home,172
4,03,alt penedes,dona,35


In [304]:
df_repetidors.to_csv('clean_data/2020/df_repetidors.csv', index=False)

In [308]:
# Agrupar les dades per 'codi' i 'comarca', sumant els valors de 'repetixen'
df_total_repetidors = df_repetidors.groupby(['codi', 'comarca'])['repetixen'].sum().reset_index()

# Renombrar la columna 'repetixen' a 'total_repetixen'
df_total_repetidors.rename(columns={'repetixen': 'total_repetixen'}, inplace=True)
df_total_repetidors.head()

,codi,comarca,total_repetixen
0,01,alt camp,101
1,02,alt emporda,300
2,03,alt penedes,155
3,04,alt urgell,14
4,05,alta ribagorca,1


In [ ]:
df_pct_ratio_estudiants.head()

In [ ]:
# Fusionar els DataFrames 'df_total_repetidors' i 'df_pct_ratio_estudiants' utilitzant la columna comuna 'codi'
df_total_repetidors_merged = df_total_repetidors.merge(df_pct_ratio_estudiants[['codi', 'poblacio_estudiant']], on='codi', how='left')

# Seleccionar només les columnes necessàries del DataFrame resultant
df_total_repetidors_merged = df_total_repetidors_merged[['codi', 'comarca', 'total_repetixen', 'poblacio_estudiant']].copy()
df_total_repetidors_merged.head()

In [ ]:
# Calcular el percentatge de repetidors respecte a la població estudiant
df_total_repetidors_merged['pct_repetidors'] = (df_total_repetidors_merged['total_repetixen'] / df_total_repetidors_merged['poblacio_estudiant'] * 100).round(2)

# Calcular la ràtio de repetidors respecte a la població estudiant
df_total_repetidors_merged['ratio_repetidors'] = (df_total_repetidors_merged['total_repetixen'] / df_total_repetidors_merged['poblacio_estudiant'])
df_total_repetidors_merged.head()

In [ ]:
df_total_repetidors_merged.to_csv('clean_data/2020/df_total_repetidors.csv', index=False)

In [310]:
df_repetidors.head()

,codi,comarca,sexe,repetixen
0,01,alt camp,dona,27
1,01,alt camp,home,74
2,02,alt emporda,dona,128
3,02,alt emporda,home,172
4,03,alt penedes,dona,35


In [312]:
df_total_repetidors_merged.head()

NameError: name 'df_total_repetidors_merged' is not defined

In [ ]:
# Fusionar 'df_repetidors' amb 'df_total_repetidors_merged' per afegir 'comarca' i 'poblacio_estudiant'
df_repetidors_genere_merged = df_repetidors.merge(
    df_total_repetidors_merged[['codi', 'comarca', 'poblacio_estudiant']], 
    on='codi', 
    how='left'
)
df_repetidors_genere_merged.head()

In [ ]:
# Eliminar 'comarca_y' i renombrar 'comarca_x' a 'comarca'
df_repetidors_genere_merged = df_repetidors_genere_merged.drop(columns=['comarca_y']).rename(columns={'comarca_x': 'comarca'})

# Calcular el total de repetidors i el total d'estudiants per comarca
df_total_repetidors = df_repetidors_genere_merged.groupby(['codi', 'comarca'])[['repetixen', 'poblacio_estudiant']].sum().reset_index()
df_total_repetidors.rename(columns={'repetixen': 'total_repetixen', 'poblacio_estudiant': 'total_estudiants'}, inplace=True)

# Afegir els totals al DataFrame original i mantenir 'comarca'
df_repetidors_genere_merged = df_repetidors_genere_merged.merge(
    df_total_repetidors[['codi', 'comarca', 'total_repetixen', 'total_estudiants']], 
    on=['codi', 'comarca'], 
    how='left'
)

df_repetidors_genere_merged.head()

In [ ]:
# Calcular el percentatge de repetidors per dones
df_repetidors_genere_merged['pct_repetidors_dones'] = (
    df_repetidors_genere_merged.loc[df_repetidors_genere_merged['sexe'] == 'dona', 'repetixen'] /
    df_repetidors_genere_merged['total_estudiants'] * 100
).fillna(0).round(2)

# Calcular el percentatge de repetidors per homes
df_repetidors_genere_merged['pct_repetidors_homes'] = (
    df_repetidors_genere_merged.loc[df_repetidors_genere_merged['sexe'] == 'home', 'repetixen'] /
    df_repetidors_genere_merged['total_estudiants'] * 100
).fillna(0).round(2)

# Calcular la ràtio de repetidors per dones
df_repetidors_genere_merged['ratio_repetidors_dones'] = (
    df_repetidors_genere_merged.loc[df_repetidors_genere_merged['sexe'] == 'dona', 'repetixen'] /
    df_repetidors_genere_merged['total_estudiants']
).fillna(0)

# Calcular la ràtio de repetidors per homes
df_repetidors_genere_merged['ratio_repetidors_homes'] = (
    df_repetidors_genere_merged.loc[df_repetidors_genere_merged['sexe'] == 'home', 'repetixen'] /
    df_repetidors_genere_merged['total_estudiants']
).fillna(0)

# Seleccionar només les columnes necessàries i eliminar duplicats
df_repetidors_poblacio = df_repetidors_genere_merged[['codi', 'comarca', 'total_repetixen', 
                                         'pct_repetidors_dones', 'pct_repetidors_homes', 
                                         'ratio_repetidors_dones', 'ratio_repetidors_homes']].drop_duplicates()

# Mostrar el DataFrame resultant
df_repetidors_poblacio.head()


In [ ]:
# Agrupar per comarca i codi per sumar els valors de dones i homes
df_repetidors_poblacio = df_repetidors_poblacio.groupby(['codi', 'comarca'], as_index=False).sum()

df_repetidors_poblacio.head()

In [ ]:
df_repetidors_poblacio.to_csv('clean_data/2020/df_repetidors_poblacio.csv', index=False)

### Nivell Socioeconòmic (Renda Familiar)<a class="anchor" id="renta_familiar"></a>

In [322]:
df_renta_familiar.head()

,Unnamed: 0,RFDB (miles de euros),RFDB por habitante (euros),RFDB por habitante (Índex Catalunya=100 por habitante)
0,Alt Camp,700228,15550,"88,6"
1,Alt Empordà,1949301,13934,"79,4"
2,Alt Penedès,1831259,16626,"94,7"
3,Alt Urgell,276359,13732,"78,3"
4,Alta Ribagorça,57401,14665,"83,6"


In [324]:
# Renombrar la columna 'Unnamed: 0' a 'comarca'
df_renta_familiar = df_renta_familiar.rename(columns={'Unnamed: 0': 'comarca'})
df_renta_familiar.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51 entries, 0 to 50
Data columns (total 4 columns):
 #   Column                                                  Non-Null Count  Dtype 
---  ------                                                  --------------  ----- 
 0   comarca                                                 51 non-null     object
 1   RFDB (miles de euros)                                   51 non-null     int64 
 2   RFDB por habitante (euros)                              51 non-null     int64 
 3   RFDB por habitante (Índex Catalunya=100 por habitante)  51 non-null     object
dtypes: int64(2), object(2)
memory usage: 1.7+ KB


In [326]:
# Netejar el DataFrame 'df_renta_familiar' utilitzant la funció 'clean_dataframe'
df_renta_familiar_clean = clean_dataframe(df_renta_familiar)
# Filtrar les dades per obtenir només les comarques rellevants
df_renta_familiar_clean = filtrar_comarques(df_renta_familiar_clean, 'comarca')
df_renta_familiar_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 4 columns):
 #   Column                                                  Non-Null Count  Dtype 
---  ------                                                  --------------  ----- 
 0   comarca                                                 42 non-null     object
 1   rfdb_(miles_de_euros)                                   42 non-null     int64 
 2   rfdb_por_habitante_(euros)                              42 non-null     int64 
 3   rfdb_por_habitante_(index_catalunya=100_por_habitante)  42 non-null     object
dtypes: int64(2), object(2)
memory usage: 1.4+ KB


In [328]:
# Afegim la columna 'codi' des de df_comarques per fer match amb els noms de les comarques
df_renta_familiar_clean.insert(0, 'codi', df_renta_familiar_clean['comarca'].map(comarques_dict))
# Ordenar el DataFrame per la columna 'codi' i reiniciar l'índex per mantenir-lo consecutiu
df_renta_familiar_clean = df_renta_familiar_clean.sort_values(by='codi').reset_index(drop=True)
df_renta_familiar_clean.head()

,codi,comarca,rfdb_(miles_de_euros),rfdb_por_habitante_(euros),rfdb_por_habitante_(index_catalunya=100_por_habitante)
0,01,alt camp,700228,15550,"88,6"
1,02,alt emporda,1949301,13934,"79,4"
2,03,alt penedes,1831259,16626,"94,7"
3,04,alt urgell,276359,13732,"78,3"
4,05,alta ribagorca,57401,14665,"83,6"


In [330]:
# Convertir els valors de la columna a format numèric
df_renta_familiar_clean['rfdb_por_habitante_(index_catalunya=100_por_habitante)'] = (
    df_renta_familiar_clean['rfdb_por_habitante_(index_catalunya=100_por_habitante)']
    .str.replace(',', '.')  # Substituir les comes per punts
    .astype(float)          # Convertir els valors a tipus float
)
df_renta_familiar_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 5 columns):
 #   Column                                                  Non-Null Count  Dtype  
---  ------                                                  --------------  -----  
 0   codi                                                    42 non-null     object 
 1   comarca                                                 42 non-null     object 
 2   rfdb_(miles_de_euros)                                   42 non-null     int64  
 3   rfdb_por_habitante_(euros)                              42 non-null     int64  
 4   rfdb_por_habitante_(index_catalunya=100_por_habitante)  42 non-null     float64
dtypes: float64(1), int64(2), object(2)
memory usage: 1.8+ KB


In [332]:
df_renta_familiar_clean.head()

,codi,comarca,rfdb_(miles_de_euros),rfdb_por_habitante_(euros),rfdb_por_habitante_(index_catalunya=100_por_habitante)
0,01,alt camp,700228,15550,88.6
1,02,alt emporda,1949301,13934,79.4
2,03,alt penedes,1831259,16626,94.7
3,04,alt urgell,276359,13732,78.3
4,05,alta ribagorca,57401,14665,83.6


In [334]:
df_renta_familiar_clean.to_csv('clean_data/2020/df_renta_familiar.csv', index=False)

### Nivell Socioeconòmic (RFBD)<a class="anchor" id="rfbd"></a>

In [344]:
df_rfbd.head()

,Unnamed: 0,Recursos. Remuneración asalariados,Recursos. Excedente bruto explotación,Recursos. Prestaciones sociales,Usos. Cotizaciones sociales,Usos. Impuestos
0,Alt Camp,549547,158967,237283,170612,90319
1,Alt Empordà,1451220,590288,603165,481126,297021
2,Alt Penedès,1514625,432157,585561,461966,260083
3,Alt Urgell,183086,80550,96098,61099,33868
4,Alta Ribagorça,41550,19002,18379,14397,7793


In [346]:
# Renombrar la columna 'Unnamed: 0' a 'comarca'
df_rfbd = df_rfbd.rename(columns={'Unnamed: 0': 'comarca'})
df_rfbd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51 entries, 0 to 50
Data columns (total 6 columns):
 #   Column                                 Non-Null Count  Dtype 
---  ------                                 --------------  ----- 
 0   comarca                                51 non-null     object
 1   Recursos. Remuneración asalariados     51 non-null     int64 
 2   Recursos. Excedente bruto explotación  51 non-null     int64 
 3   Recursos. Prestaciones sociales        51 non-null     int64 
 4   Usos. Cotizaciones sociales            51 non-null     int64 
 5   Usos. Impuestos                        51 non-null     int64 
dtypes: int64(5), object(1)
memory usage: 2.5+ KB


In [348]:
# Netejar el DataFrame 'df_renta_familiar' utilitzant la funció 'clean_dataframe'
df_rfbd_clean = clean_dataframe(df_rfbd)
# Filtrar les dades per obtenir només les comarques rellevants
df_rfbd_clean = filtrar_comarques(df_rfbd_clean, 'comarca')
df_rfbd_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 6 columns):
 #   Column                                 Non-Null Count  Dtype 
---  ------                                 --------------  ----- 
 0   comarca                                42 non-null     object
 1   recursos._remuneracion_asalariados     42 non-null     int64 
 2   recursos._excedente_bruto_explotacion  42 non-null     int64 
 3   recursos._prestaciones_sociales        42 non-null     int64 
 4   usos._cotizaciones_sociales            42 non-null     int64 
 5   usos._impuestos                        42 non-null     int64 
dtypes: int64(5), object(1)
memory usage: 2.1+ KB


In [350]:
# Afegim la columna 'codi' des de df_comarques per fer match amb els noms de les comarques
df_rfbd_clean.insert(0, 'codi', df_rfbd_clean['comarca'].map(comarques_dict))
# Ordenar el DataFrame per la columna 'codi' i reiniciar l'índex per mantenir-lo consecutiu
df_rfbd_clean = df_rfbd_clean.sort_values(by='codi').reset_index(drop=True)
df_rfbd_clean.head()

,codi,comarca,recursos._remuneracion_asalariados,recursos._excedente_bruto_explotacion,recursos._prestaciones_sociales,usos._cotizaciones_sociales,usos._impuestos
0,01,alt camp,549547,158967,237283,170612,90319
1,02,alt emporda,1451220,590288,603165,481126,297021
2,03,alt penedes,1514625,432157,585561,461966,260083
3,04,alt urgell,183086,80550,96098,61099,33868
4,05,alta ribagorca,41550,19002,18379,14397,7793


In [352]:
df_rfbd_clean.to_csv('clean_data/2020/df_rfbd.csv', index=False)